In [1]:
#Import the required libraries
!pip install keras-tuner
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import pathlib, os, random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow as tf
from PIL import Image
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import time
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


import cv2
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from PIL import Image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.utils import np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.9 MB/s 
     |████████████████████████████████| 1.6 MB 40.0 MB/s 


In [15]:
#Understanding the files in different data directories
bad_data = os.listdir('/content/drive/MyDrive/Ethiopia_3/Bad')
print("Bad coffee beans: ",bad_data[:10])

good_data = os.listdir('/content/drive/MyDrive/Ethiopia_3/Good')
print("Good coffee beans: ",good_data[:10])

Bad coffee beans:  ['IMG_5985_1.jpg', 'IMG_5983_1.jpg', 'IMG_5980_1.jpg', 'IMG_5979_1.jpg', 'IMG_5981_7.jpg', 'IMG_5979_7.jpg', 'IMG_5984_3.jpg', 'IMG_5979_9.jpg', 'IMG_5982_4.jpg', 'IMG_5985_6.jpg']
Good coffee beans:  ['IMG_5923_8.jpg', 'IMG_5919_8.jpg', 'IMG_5918_6.jpg', 'IMG_5923_7.jpg', 'IMG_5922_2.jpg', 'IMG_5920_5.jpg', 'IMG_5921_9.jpg', 'IMG_5918_3.jpg', 'IMG_5923_6.jpg', 'IMG_5917_9.jpg']


In [16]:
data = []
labels = []
for img in bad_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia_3/Bad' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(1)
    except:
        None
        
for img in good_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia_3/Good' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(0)
    except:
        None

In [17]:
image_data = np.array(data)
labels = np.array(labels)

idx = np.arange(image_data.shape[0])
np.random.shuffle(idx)
image_data = image_data[idx]
labels = labels[idx]

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 101)

y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

print(f'SHAPE OF TRAINING IMAGE DATA : {x_train.shape}')
print(f'SHAPE OF TESTING IMAGE DATA : {x_test.shape}')
print(f'SHAPE OF TRAINING LABELS : {y_train.shape}')
print(f'SHAPE OF TESTING LABELS : {y_test.shape}')

SHAPE OF TRAINING IMAGE DATA : (112, 256, 256, 3)
SHAPE OF TESTING IMAGE DATA : (28, 256, 256, 3)
SHAPE OF TRAINING LABELS : (112, 2)
SHAPE OF TESTING LABELS : (28, 2)


In [19]:
!pip install hyperactive==0.3.1
import hyperactive as hyp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
search_config = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [16, 32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": range(30, 100, 10), "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": list(np.arange(0.2, 0.8, 0.2))},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}
start_point = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [64],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [32],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": [30], "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": [0.2]},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}


#https://pypi.org/project/hyperactive/0.3.1/ 
Optimizer = hyp.ParticleSwarmOptimizer(search_config, n_iter=3,warm_start=start_point, metric="accuracy", n_jobs=1, cv=3, verbosity=1, random_state=42, n_part=3, w=0.85, c_k=1.9, c_s=2.05)

In [21]:
import time
t1 = time.time()
# search best hyperparameter for given data
h = Optimizer.fit(x_train, y_train)

t2 = time.time()
print("time: {}".format(t2-t1))

# predict from test data
Optimizer.predict(x_test)

# calculate accuracy score
score = Optimizer.score(x_test, y_test)

print("test score: {}".format(score))

Epoch 1/5
4/4 - 2s - loss: 851.6553 - accuracy: 0.4821 - 2s/epoch - 477ms/step
Epoch 2/5
4/4 - 0s - loss: 34.8640 - accuracy: 0.5179 - 253ms/epoch - 63ms/step
Epoch 3/5
4/4 - 0s - loss: 5.4423 - accuracy: 0.5268 - 253ms/epoch - 63ms/step
Epoch 4/5
4/4 - 0s - loss: 3.4298 - accuracy: 0.4732 - 252ms/epoch - 63ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6931 - accuracy: 0.5000 - 244ms/epoch - 61ms/step
4/4 [==============================] - 0s 31ms/step - loss: 0.6931 - accuracy: 0.5000


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5
4/4 - 1s - loss: 551.7710 - accuracy: 0.4107 - 1s/epoch - 313ms/step
Epoch 2/5
4/4 - 0s - loss: 111.3495 - accuracy: 0.4554 - 151ms/epoch - 38ms/step
Epoch 3/5
4/4 - 0s - loss: 13.5442 - accuracy: 0.5357 - 153ms/epoch - 38ms/step
Epoch 4/5
4/4 - 0s - loss: 0.8538 - accuracy: 0.6786 - 158ms/epoch - 40ms/step
Epoch 5/5
4/4 - 0s - loss: 0.4490 - accuracy: 0.7946 - 145ms/epoch - 36ms/step
4/4 [==============================] - 0s 18ms/step - loss: 0.2718 - accuracy: 0.9375
Epoch 1/5
4/4 - 1s - loss: 154.0101 - accuracy: 0.5179 - 1s/epoch - 316ms/step
Epoch 2/5
4/4 - 0s - loss: 7.4298 - accuracy: 0.4732 - 267ms/epoch - 67ms/step
Epoch 3/5
4/4 - 0s - loss: 0.6893 - accuracy: 0.4732 - 294ms/epoch - 73ms/step
Epoch 4/5
4/4 - 0s - loss: 0.6652 - accuracy: 0.4821 - 264ms/epoch - 66ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6035 - accuracy: 0.5804 - 250ms/epoch - 63ms/step
4/4 [==============================] - 0s 40ms/step - loss: 0.5009 - accuracy: 0.5804
Epoch 1/5
4/4 - 4s - loss: 675.4406

 33%|███▎      | 1/3 [00:12<00:24, 12.42s/it]

Epoch 1/5
4/4 - 1s - loss: 1388.8821 - accuracy: 0.3929 - 991ms/epoch - 248ms/step
Epoch 2/5
4/4 - 0s - loss: 168.9394 - accuracy: 0.5446 - 206ms/epoch - 51ms/step
Epoch 3/5
4/4 - 0s - loss: 11.0860 - accuracy: 0.5625 - 195ms/epoch - 49ms/step
Epoch 4/5
4/4 - 0s - loss: 22.4843 - accuracy: 0.5000 - 199ms/epoch - 50ms/step
Epoch 5/5
4/4 - 0s - loss: 0.6917 - accuracy: 0.5089 - 169ms/epoch - 42ms/step
4/4 [==============================] - 0s 25ms/step - loss: 14.6814 - accuracy: 0.5000
Epoch 1/5
4/4 - 1s - loss: 634.8524 - accuracy: 0.5357 - 952ms/epoch - 238ms/step
Epoch 2/5
4/4 - 0s - loss: 41.1516 - accuracy: 0.5625 - 147ms/epoch - 37ms/step
Epoch 3/5
4/4 - 0s - loss: 6.2224 - accuracy: 0.6339 - 152ms/epoch - 38ms/step
Epoch 4/5
4/4 - 0s - loss: 0.3344 - accuracy: 0.8839 - 152ms/epoch - 38ms/step
Epoch 5/5
4/4 - 0s - loss: 0.2737 - accuracy: 0.9732 - 145ms/epoch - 36ms/step
4/4 [==============================] - 0s 20ms/step - loss: 0.1302 - accuracy: 0.9911
Epoch 1/5
4/4 - 1s - loss

 67%|██████▋   | 2/3 [00:19<00:09,  9.22s/it]

Epoch 1/5
4/4 - 1s - loss: 215.0172 - accuracy: 0.4911 - 631ms/epoch - 158ms/step
Epoch 2/5
4/4 - 0s - loss: 46.2855 - accuracy: 0.5804 - 154ms/epoch - 39ms/step
Epoch 3/5
4/4 - 0s - loss: 22.9610 - accuracy: 0.6250 - 150ms/epoch - 37ms/step
Epoch 4/5
4/4 - 0s - loss: 0.5403 - accuracy: 0.8304 - 150ms/epoch - 37ms/step
Epoch 5/5
4/4 - 0s - loss: 0.3202 - accuracy: 0.8571 - 145ms/epoch - 36ms/step
4/4 [==============================] - 0s 20ms/step - loss: 0.2488 - accuracy: 0.9107
Epoch 1/5
4/4 - 1s - loss: 491.5859 - accuracy: 0.4554 - 656ms/epoch - 164ms/step
Epoch 2/5
4/4 - 0s - loss: 31.7766 - accuracy: 0.5625 - 153ms/epoch - 38ms/step
Epoch 3/5
4/4 - 0s - loss: 7.8497 - accuracy: 0.5804 - 153ms/epoch - 38ms/step
Epoch 4/5
4/4 - 0s - loss: 0.7237 - accuracy: 0.8750 - 158ms/epoch - 39ms/step
Epoch 5/5
4/4 - 0s - loss: 0.3733 - accuracy: 0.8393 - 144ms/epoch - 36ms/step
4/4 [==============================] - 0s 19ms/step - loss: 0.1626 - accuracy: 0.9464
Epoch 1/5
4/4 - 1s - loss: 75


 accuracy 0.9910714030265808
start_point = {'keras.compile.0': {'loss': ['binary_crossentropy'], 'optimizer': ['adam']}, 'keras.fit.0': {'epochs': [5], 'batch_size': [32], 'verbose': [2]}, 'keras.layers.Conv2D.1': {'filters': [32], 'kernel_size': [(3, 3)], 'activation': ['relu'], 'input_shape': [(256, 256, 3)]}, 'keras.layers.MaxPooling2D.2': {'pool_size': [(2, 2)]}, 'keras.layers.Conv2D.3': {'filters': [16], 'kernel_size': [(3, 3)], 'activation': ['relu']}, 'keras.layers.MaxPooling2D.4': {'pool_size': [(2, 2)]}, 'keras.layers.Flatten.5': {}, 'keras.layers.Dense.6': {'units': [80], 'activation': ['relu']}, 'keras.layers.Dropout.7': {'rate': [0.2]}, 'keras.layers.Dense.8': {'units': [2], 'activation': ['softmax']}}
1/4 [======>.......................] - ETA: 0s - loss: 0.6932 - accuracy: 0.4688

4/4 [==============================] - 0s 62ms/step - loss: 0.6931 - accuracy: 0.5000
time: 29.007092475891113
1/1 [==============================] - 0s 50ms/step - loss: 0.6931 - accuracy: 0.5000
test score: 0.5
